# Data augmentation 

A seguir, estamos combinando o código gerado nas sprints anteriores com o processo de tratamento e pré processamento de imagens através da algoritmos,  dos pixels e com o processo de Data Augmentation, que foi atualizado desde a sprint 1. Removemos alguns filtros e implementamos códigos necessários para garantir que o conjunto de imagens seja eficaz.

- [Pré Processamento de imagens utilizando Kmeans e Sharpen - SPRINT 1](../../SPRINT%201/20240425%20-%20Implementacao%20KMeans%20para%20Reducao%20de%20Cor.ipynb)
- [Data Augmentation SPRINT 3](../../SPRINT%203/DATA%20AUGMENTATION/20240524%20-%20Data%20Augmentation.ipynb)

In [ ]:
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
import tqdm
from google.colab import drive
from PIL import Image

# Mount Google Drive
drive.mount('/content/drive')

In [ ]:

# Função auxiliar para criação de diretórios através de um caminho
def create_dirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:


class KMeansImageProcessingPipeline:
    def __init__(self, root_dir, output_dir, k=3, attempts=10):
        self.root_dir = root_dir
        self.output_dir = output_dir
        self.k = k
        self.attempts = attempts
        self.target_size = (600, 600)

    def read_and_process_image(self, path):
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        if img.shape[:2] != self.target_size:
            img = cv2.resize(img, self.target_size[::-1])
        return img

    def kmeans_image(self, img):
        # Redimensionar a imagem para um vetor 1D
        vectorized_img = img.reshape((-1, 3))
        vectorized_img = np.float32(vectorized_img)

        # Definir critérios para o algoritmo K-means
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)

        # Executar o algoritmo K-means
        _, label, center = cv2.kmeans(vectorized_img, self.k, None, criteria, self.attempts, cv2.KMEANS_PP_CENTERS)
        center = np.uint8(center)

        # Recriar a imagem a partir dos clusters encontrados
        res = center[label.flatten()]
        result_image = res.reshape((img.shape))

        return result_image

    def sharpen_image(self, img):
        # Cria um kernel para afiar a imagem
        kernel = np.array([[-1, -1, -1],
                           [-1, 9, -1],
                           [-1, -1, -1]])

        # Aplica o kernel na imagem usando filter2D
        sharpened_image = cv2.filter2D(img, -1, kernel)

        return sharpened_image

    def process_directory(self):
        create_dirs(self.output_dir)
        processed_images = []
        for subdir, _, files in os.walk(self.root_dir):
            tif_files = sorted([os.path.join(subdir, file) for file in files if file.endswith('_TCI.tif')])
            if tif_files:
                images = [(file, self.read_and_process_image(file)) for file in tif_files if self.read_and_process_image(file) is not None]
                if images:
                    for file, image in images:
                        kmeans_img = self.kmeans_image(image)
                        sharpened_img = self.sharpen_image(kmeans_img)
                        # Extrai o primeiro número antes do _ do nome do arquivo original
                        image_name = os.path.basename(file).split('_')[0]
                        # Converte para PIL e salva no diretório de saída
                        pil_img = Image.fromarray((sharpened_img).astype(np.uint8))
                        pil_img.save(os.path.join(self.output_dir, f"{image_name}.png"))
                        processed_images.append(sharpened_img)
        return processed_images

# Paths
root_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/tci_tifs'
output_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/kmeans_processed_images'

# Process images and save them
pipeline = KMeansImageProcessingPipeline(root_dir, output_dir)
processed_images = pipeline.process_directory()


In [ ]:
plt.imshow(processed_images[13])
plt.show()

In [ ]:
import os
import cv2

# Function to load masks
def load_masks(masks_dir, target_size=(600, 600)):
    masks = []
    ordered_masks = sorted(os.listdir(masks_dir))
    mask_filenames = []
    for mask_name in ordered_masks:
        mask_path = os.path.join(masks_dir, mask_name)
        mask = cv2.imread(mask_path, cv2.IMREAD_UNCHANGED)
        if mask is not None:
            if mask.shape[:2] != target_size:
                mask = cv2.resize(mask, target_size[::-1])
            masks.append(mask)
            mask_filenames.append(os.path.splitext(mask_name)[0])
        else:
            print(f"Failed to load mask: {mask_path}")
    return masks, mask_filenames

# Function to load images
def load_images(image_dir, target_size=(600, 600)):
    images = []
    ordered_images = sorted(os.listdir(image_dir))
    image_filenames = []
    for image_name in ordered_images:
        image_path = os.path.join(image_dir, image_name)
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is not None:
            if image.shape[:2] != target_size:
                image = cv2.resize(image, target_size[::-1])
            images.append(image)
            # Extract the first segment of the filename before the underscore
            image_filenames.append(os.path.splitext(image_name)[0].split('_')[0])
        else:
            print(f"Failed to load image: {image_path}")
    return images, image_filenames

# Paths
masks_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/masks'
image_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/tci_tifs'

# Load images and masks
masks, mask_filenames = load_masks(masks_dir)
images, image_filenames = load_images(image_dir)


In [ ]:
# Visualizando iamgem
plt.imshow(images[3])
plt.show()

In [ ]:
# Garante que as máscaras e imagens estão na mesma ordem
assert mask_filenames == image_filenames, "A ordem das máscaras e imagens não coincide!"

class ImageProcessingPipeline:
    def __init__(self, images, masks):
        # Inicializa a classe com as imagens e máscaras
        self.images = images
        self.masks = masks

    def crop_image(self, image, crop_size=(200, 200)):
        # Método para recortar uma imagem em pedaços menores
        crops = []
        for i in range(0, image.shape[0], crop_size[0]):
            for j in range(0, image.shape[1], crop_size[1]):
                crop = image[i:i+crop_size[0], j:j+crop_size[1]]
                if crop.shape[0] == crop_size[0] and crop.shape[1] == crop_size[1]:
                    crops.append(crop)
        return crops

    def augment_images(self, image):
        # Método para aumentar as imagens (rotação e espelhamento)
        aug_images = []
        for angle in [0, 90, 180, 270]:
            rotated = self.rotate_image(image, angle)
            aug_images.append(rotated)
            aug_images.append(cv2.flip(rotated, 1))
        return aug_images

    @staticmethod
    def rotate_image(image, angle):
        # Método para rotacionar uma imagem
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        return cv2.warpAffine(image, M, (w, h))

    def process_and_save_images_and_masks(self, output_dir):
        # Processa e salva as imagens e máscaras em um diretório de saída
        create_dirs(output_dir)
        count = 0
        for img, mask in zip(self.images, self.masks):
            cropped_images = self.crop_image(img)
            cropped_masks = self.crop_image(mask)
            for crop_img, crop_mask in zip(cropped_images, cropped_masks):
                augmented_imgs = self.augment_images(crop_img)
                augmented_masks = self.augment_images(crop_mask)
                for aug_img, aug_mask in zip(augmented_imgs, augmented_masks):
                    aug_img = Image.fromarray((aug_img * 255).astype(np.uint8))  # Converte de volta para uint8
                    aug_mask = Image.fromarray(aug_mask)
                    output_img_path = os.path.join(output_dir, f'processed_image_{count}.tif')
                    output_mask_path = os.path.join(output_dir, f'processed_mask_{count}.png')
                    print(f"Salvando imagem processada em {output_img_path}")
                    print(f"Salvando máscara processada em {output_mask_path}")
                    aug_img.save(output_img_path)
                    aug_mask.save(output_mask_path)
                    count += 1

    def show_image(self, image):
        # Método para exibir uma imagem
        plt.imshow(image, cmap='gray')
        plt.axis('off')
        plt.show()

# Processa e aumenta imagens e máscaras
output_dir = '/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/clean_dataset'
pipeline = ImageProcessingPipeline(images, masks)
pipeline.process_and_save_images_and_masks(output_dir)

# Mostra algumas das imagens processadas
for img in images[:8]:  # Mostra as primeiras 8 imagens processadas
    pipeline.show_image(img)

Caso haja necessidade de executar o código, devido ao peso dos datasets optamos por mante-los no google drive, contudo podem ser acessados através dos links abaixo:

- [Conjunto de imagens geradas através de préprocessamento com KMeans, Sharpen e data augmentation ](https://drive.google.com/drive/folders/18V4LmUwD1im1LuOFRuzKta6hUgAZr5AW?usp=sharing)